In [ ]:
import pandas as pd

In [ ]:
weather_data = pd.read_csv('Resources/PostgreSQL_weatherdata.csv')  
weather_data.head()

In [ ]:
weather_data = weather_data.drop(columns=['wind_mph', 'cloud', 'uv_index'])

In [ ]:
weather_data.head()

In [ ]:
# Map without Search bar 

import ipywidgets as widgets
from IPython.display import display
import folium
from folium.plugins import Fullscreen, MiniMap
import pandas as pd



weather_data['last_updated'] = pd.to_datetime(weather_data['last_updated'])
weather_data['month'] = weather_data['last_updated'].dt.strftime('%B %Y')  # Format: "Month YYYY"

month_dropdown = widgets.Dropdown(
    options=weather_data['month'].unique(),  # List of unique months (formatted)
    value=weather_data['month'].unique()[0],  # Default value
    description='Date:',
)

def update_map(change):
    selected_month = month_dropdown.value
    filtered_gdf = weather_data[weather_data['month'] == selected_month]  # Filter data by month

    weather_map = folium.Map(
        location=[weather_data['latitude'].mean(), weather_data['longitude'].mean()],
        zoom_start=4
    )
    Fullscreen().add_to(weather_map)
    MiniMap(toggle_display=True).add_to(weather_map)

    for _, row in filtered_gdf.iterrows():
        color = 'red' if row['temperature_fahrenheit'] > 80 else 'blue'
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Country:</b> {row['country']}<br>"
                  f"<b>Location:</b> {row['location_name']}<br>"
                  f"<b>Temperature:</b> {row['temperature_fahrenheit']}°F<br>"
                  f"<b>Humidity:</b> {row['humidity']}%",
            icon=folium.Icon(color=color, icon='cloud'),
        ).add_to(weather_map)

  
    legend_html = """
    <div style="
        position: fixed; 
        bottom: 50px; left: 50px; width: 120px; height: auto; 
        background: linear-gradient(to bottom, #ffffff, #f7f7f7); 
        border-radius: 10px; padding: 15px; 
        box-shadow: 0px 3px 6px rgba(0, 0, 0, 0.2); 
        font-family: Arial, sans-serif; font-size: 14px; 
        color: #333; line-height: 1.6; z-index:1000;">
        <b style="font-size: 16px;">Temperature</b><br>
        <div style="display: flex; align-items: center; margin-top: 5px;">
            <div style="width: 15px; height: 15px; background: red; border-radius: 50%; margin-right: 10px;"></div>
            <span> > 80°F</span>
        </div>
        <div style="display: flex; align-items: center; margin-top: 5px;">
            <div style="width: 15px; height: 15px; background: blue; border-radius: 50%; margin-right: 10px;"></div>
            <span> ≤ 80°F</span>
        </div>
    </div>
    """
    weather_map.get_root().html.add_child(folium.Element(legend_html))

    title_html = f"""
        <h3 align="center" style="font-size:15px"><b>Weather Map - {selected_month}</b></h3>
    """
    weather_map.get_root().html.add_child(folium.Element(title_html))

    display(weather_map)

month_dropdown.observe(update_map, names='value')

display(month_dropdown)
update_map(None)


In [ ]:
# Map with search bar
import ipywidgets as widgets
from IPython.display import display
import folium
from folium.plugins import Fullscreen, MiniMap, Search
import pandas as pd


weather_data['last_updated'] = pd.to_datetime(weather_data['last_updated'])
weather_data['month'] = weather_data['last_updated'].dt.strftime('%B %Y')  

month_dropdown = widgets.Dropdown(
    options=weather_data['month'].unique(),  
    value=weather_data['month'].unique()[0],  
    description='Date:',
)

def update_map(change):
    selected_month = month_dropdown.value
    filtered_gdf = weather_data[weather_data['month'] == selected_month]  # Filter data by month

    weather_map = folium.Map(
        location=[weather_data['latitude'].mean(), weather_data['longitude'].mean()],
        zoom_start=4
    )
    Fullscreen().add_to(weather_map)
    MiniMap(toggle_display=True).add_to(weather_map)

    feature_group = folium.FeatureGroup(name="Weather Markers").add_to(weather_map)

    for _, row in filtered_gdf.iterrows():
        color = 'red' if row['temperature_fahrenheit'] > 80 else 'blue'
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>Country:</b> {row['country']}<br>"
                  f"<b>Location:</b> {row['location_name']}<br>"
                  f"<b>Temperature:</b> {row['temperature_fahrenheit']}°F<br>"
                  f"<b>Humidity:</b> {row['humidity']}%",
            icon=folium.Icon(color=color, icon='cloud'),
        ).add_to(feature_group)

    search = Search(
        layer=feature_group,  
        search_label="location_name",  
        placeholder="Search for a location",  
        position="topright",  
        collapsed=False,  
    )
    search.add_to(weather_map)

   
    legend_html = """
    <div style="
        position: fixed; 
        bottom: 50px; left: 50px; width: 120px; height: auto; 
        background: linear-gradient(to bottom, #ffffff, #f7f7f7); 
        border-radius: 10px; padding: 15px; 
        box-shadow: 0px 3px 6px rgba(0, 0, 0, 0.2); 
        font-family: Arial, sans-serif; font-size: 14px; 
        color: #333; line-height: 1.6; z-index:1000;">
        <b style="font-size: 16px;">Temperature</b><br>
        <div style="display: flex; align-items: center; margin-top: 5px;">
            <div style="width: 15px; height: 15px; background: red; border-radius: 50%; margin-right: 10px;"></div>
            <span>80°F</span>
        </div>
        <div style="display: flex; align-items: center; margin-top: 5px;">
            <div style="width: 15px; height: 15px; background: dodgerblue; border-radius: 50%; margin-right: 10px;"></div>
            <span>≤ 80°F</span>
        </div>
    </div>
    """
    weather_map.get_root().html.add_child(folium.Element(legend_html))

    title_html = f"""
        <h3 align="center" style="font-size:15px"><b>Weather Map - {selected_month}</b></h3>
    """
    weather_map.get_root().html.add_child(folium.Element(title_html))

    display(weather_map)

month_dropdown.observe(update_map, names='value')

display(month_dropdown)
update_map(None)
